In [26]:
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import linear_model
from sklearn import svm

In [2]:
train = pd.read_excel('train.xlsx',sheetname=0)
test = pd.read_excel('test.xlsx',sheetname=0)
(n,p) =train.shape
(m,p)=test.shape

In [3]:
# convert Gender into dummy columnes
gen = pd.get_dummies(pd.concat([train['Gender'],test['Gender']],axis=0))
b10 = pd.get_dummies(pd.concat([train['10board'],test['10board']],axis=0))
deg = pd.get_dummies(pd.concat([train['Degree'],test['Degree']],axis=0))
spec = pd.get_dummies(pd.concat([train['Specialization'],test['Specialization']],axis=0))
gen.shape

(5498, 2)

In [4]:
#drop redundant and unused columns
X = train.drop(['ID','Salary','DOJ','DOL','Designation','JobCity','Gender','DOB','10board','12board','Degree','Specialization','CollegeState','CollegeID','CollegeCityID'],axis=1)

In [5]:

X.head()


,10percentage,12graduation,12percentage,CollegeTier,collegeGPA,CollegeCityTier,GraduationYear,English,Logical,Quant,Domain,ComputerProgramming,ElectronicsAndSemicon,ComputerScience,MechanicalEngg,ElectricalEngg,TelecomEngg,CivilEngg,conscientiousness,agreeableness,
train,84.3,2007,95.8,2,78.00,0,2011,515,585,525,0.635979,445,-1,-1,-1,-1,-1,-1,0.9737,0.8128,...
train,85.4,2007,85.0,2,70.06,0,2012,695,610,780,0.960603,-1,466,-1,-1,-1,-1,-1,-0.7335,0.3789,...
train,85.0,2010,68.2,2,70.00,0,2014,615,545,370,0.450877,395,-1,-1,-1,-1,-1,-1,0.2718,1.7109,...
train,85.6,2007,83.6,1,74.64,1,2011,635,585,625,0.974396,615,-1,-1,-1,-1,-1,-1,0.0464,0.3448,...
train,78.0,2008,76.8,2,73.90,0,2012,545,625,465,0.124502,-1,233,-1,-1,-1,-1,-1,-0.8810,-0.2793,...


In [6]:
X.ComputerProgramming = preprocessing.scale(train['ComputerProgramming'].replace(-1,None))
X.ElectronicsAndSemicon = preprocessing.scale(train['ElectronicsAndSemicon'].replace(-1,None))
X.ComputerScience = preprocessing.scale(train['ComputerScience'].replace(-1,None))
X.MechanicalEngg = preprocessing.scale(train['MechanicalEngg'].replace(-1,None))
X.ElectricalEngg = preprocessing.scale(train['ElectricalEngg'].replace(-1,None))
X.TelecomEngg = preprocessing.scale(train['TelecomEngg'].replace(-1,None))
X.CivilEngg = preprocessing.scale(train['CivilEngg'].replace(-1,None))
max_scale =X[['ComputerProgramming','ElectronicsAndSemicon','ComputerScience','MechanicalEngg','ElectricalEngg','TelecomEngg','CivilEngg']].max(axis=1)
X.Domain = X.Domain.replace(-1,max_scale)

X = X.drop(['ComputerProgramming','ElectronicsAndSemicon','ComputerScience','MechanicalEngg','ElectricalEngg','TelecomEngg','CivilEngg'],axis=1)


/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to 

In [7]:
X = pd.concat([X,gen[:n],b10[:n],deg[:n],spec[:n]],axis=1)
X.head()

,10percentage,12graduation,12percentage,CollegeTier,collegeGPA,CollegeCityTier,GraduationYear,English,Logical,Quant,Domain,conscientiousness,agreeableness,extraversion,nueroticism,openess_to_experience,f,m,0,board of secondary education,
train,84.3,2007,95.8,2,78.00,0,2011,515,585,525,0.635979,0.9737,0.8128,0.5269,1.35490,-0.4455,1,0,0,0,...
train,85.4,2007,85.0,2,70.06,0,2012,695,610,780,0.960603,-0.7335,0.3789,1.2396,-0.10760,0.8637,0,1,0,0,...
train,85.0,2010,68.2,2,70.00,0,2014,615,545,370,0.450877,0.2718,1.7109,0.1637,-0.86820,0.6721,1,0,0,0,...
train,85.6,2007,83.6,1,74.64,1,2011,635,585,625,0.974396,0.0464,0.3448,-0.3440,-0.40780,-0.9194,0,1,0,0,...
train,78.0,2008,76.8,2,73.90,0,2012,545,625,465,0.124502,-0.8810,-0.2793,-1.0697,0.09163,-0.1295,0,1,0,0,...


In [8]:

Xs = X._get_numeric_data()


In [9]:
# training for 3200 rows
X_train = Xs[:3200]
y_train = train[:3200].Salary

clf_partial = LinearRegression()
clf_partial = clf_partial.fit(X_train,y_train)


In [10]:
#testing for rest of the rows
X_test = Xs[3201:]
y_test = train[3201:].Salary

r_sqr = clf_partial.score(X_test,y_test)
y_pred = clf_partial.predict(X_test)

mae = mean_absolute_error(y_test,y_pred)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(rmse)
print(r_sqr)

162036.60222446467
0.150492074542


In [11]:
# training for the full training set
Xtrain = Xs
ytrain = train.Salary

clf = LinearRegression()
clf = clf.fit(Xtrain,ytrain)




In [12]:
#test = pd.read_excel('test.xlsx',sheetname=0)


In [13]:
# preprocessing on the test file
Xt = test.drop(['ID','Salary','DOJ','DOL','Designation','JobCity','Gender','DOB','10board','12board','Degree','Specialization','CollegeState','CollegeID','CollegeCityID'],axis=1)



Xt.ComputerProgramming = preprocessing.scale(test['ComputerProgramming'].replace(-1,None))
Xt.ElectronicsAndSemicon = preprocessing.scale(test['ElectronicsAndSemicon'].replace(-1,None))
Xt.ComputerScience = preprocessing.scale(test['ComputerScience'].replace(-1,None))
Xt.MechanicalEngg = preprocessing.scale(test['MechanicalEngg'].replace(-1,None))
Xt.ElectricalEngg = preprocessing.scale(test['ElectricalEngg'].replace(-1,None))
Xt.TelecomEngg = preprocessing.scale(test['TelecomEngg'].replace(-1,None))
Xt.CivilEngg = preprocessing.scale(test['CivilEngg'].replace(-1,None))

max_scale =Xt[['ComputerProgramming','ElectronicsAndSemicon','ComputerScience','MechanicalEngg','ElectricalEngg','TelecomEngg','CivilEngg']].max(axis=1)

Xt.Domain = Xt.Domain.replace(-1,max_scale)

Xt = Xt.drop(['ComputerProgramming','ElectronicsAndSemicon','ComputerScience','MechanicalEngg','ElectricalEngg','TelecomEngg','CivilEngg'],axis=1)

Xt = pd.concat([Xt,gen[n:],b10[n:],deg[n:],spec[n:]],axis=1)
Xt = Xt._get_numeric_data()
Xt.shape


/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to 

(1500, 429)

In [14]:
#no Salaries in test set to compare to 
Xtest = Xt
ytest = test.Salary

#r_sqr = clf.score(Xtest,ytest)
ypred = clf.predict(Xtest)

#mae = mean_absolute_error(ytest,ypred)
#rmse = sqrt(mean_squared_error(ytest, ypred))
#print(rmse)

In [15]:
# linear regression predictions saved in result1
pd.DataFrame({'ID':test.ID,'Salary':ypred}).to_excel('result1.xlsx',index=False)

In [16]:
# using svm model
sv= svm.SVR(kernel='linear')
sal_pred = Pipeline([("feat_sel",SelectKBest(k=400)),("svr",sv)])

In [17]:
#training for 3200 rows via svm pipeline
sal_pred.fit(X_train,y_train)
y_tpred = sal_pred.predict(X_train)
rmse = sqrt(mean_squared_error(y_train, y_tpred))
print(rmse)

214073.82905720675


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 19  20  22  23  25  27  29  30  32  33  39  42  43  46  52  53  58  60
  68  70  78  79  82  88  92  95  96 100 102 104 105 106 107 110 112 116
 118 122 133 138 140 141 142 146 148 149 152 158 166 170 172 173 179 180
 182 183 184 186 189 194 198 203 205 209 213 216 217 218 220 230 231 232
 239 241 245 252 254 257 260 261 262 266 268 272 275 276 279 284 288 289
 291 294 295 297 298 300 304 305 306 312 313 314 318 325 326 330 335 338
 342 345 347 348 353 355 359 360 361 364 365 374 377 379 383 388 391 395
 404 408 421 425 427 428] are constant.
  UserWarning)


In [23]:
# this test set is part of the training file
r_sqr = sal_pred.score(X_test,y_test)
y_ppred = sal_pred.predict(X_test)

mae = mean_absolute_error(y_test,y_ppred)
rmse = sqrt(mean_squared_error(y_test, y_ppred))
print(rmse)
print(r_sqr)


169667.2937931462
0.0685974018163


In [24]:
# train the full train dataset
Xtrain = Xs
ytrain = train.Salary
sal_pred.fit(Xtrain,ytrain)
y_tpred = sal_pred.predict(Xtrain)
rmse = sqrt(mean_squared_error(ytrain, y_tpred))
print(rmse)
#printing the training rmse

205842.54702164244


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 19  20  22  23  25  27  33  53  58  60  68  78  88  95 100 102 105 106
 112 116 118 122 138 141 142 146 149 152 158 166 170 173 180 183 184 186
 189 198 203 205 216 220 230 231 232 239 245 252 257 260 261 266 276 279
 284 289 291 295 297 298 300 305 306 312 313 314 318 325 326 330 335 338
 348 353 355 360 361 365 374 379 383 391 404 408 421 425 427 428] are constant.
  UserWarning)


In [20]:
#no Salaries in test set to compare to 
Xtest = Xt
ytest = test.Salary

#r_sqr = clf.score(Xtest,ytest)
ypred = sal_pred.predict(Xtest)

#mae = mean_absolute_error(ytest,ypred)
#rmse = sqrt(mean_squared_error(ytest, ypred))
#print(rmse)

In [21]:
# svm predictions saved in result2
pd.DataFrame({'ID':test.ID,'Salary':ypred}).to_excel('result2.xlsx',index=False)

In [31]:
#training for 3200 rows via lasso
las = linear_model.Ridge(alpha = 0.5)
las.fit(X_train,y_train)
y_tpred = sal_pred.predict(X_train)
rmse = sqrt(mean_squared_error(y_train, y_tpred))
print(rmse)

213930.6968870148


In [32]:
# this test set is part of the training file
r_sqr = las.score(X_test,y_test)
y_lpred = las.predict(X_test)

mae = mean_absolute_error(y_test,y_lpred)
rmse = sqrt(mean_squared_error(y_test, y_lpred))
print(rmse)
print(r_sqr)


160154.99619743132
0.170106883666
